In [3]:
# config

LABELS = ['hand']
IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = len(LABELS)


In [4]:
wt_path = 'yolo.weights'                      
train_image_folder = os.path.join(".", "data", "train", "images\\")
train_annot_folder = os.path.join(".", "data", "train", "annotations\\")



In [9]:
import random
import argparse
import numpy as np

from preprocessing import parse_annotation
import json

# argparser = argparse.ArgumentParser()

# argparser.add_argument(
#     '-c',
#     '--conf',
#     default='config.json',
#     help='path to configuration file')

# argparser.add_argument(
#     '-a',
#     '--anchors',
#     default=5,
#     help='number of anchors to use')





def IOU(ann, centroids):
    w, h = ann
    similarities = []

    for centroid in centroids:
        c_w, c_h = centroid

        if c_w >= w and c_h >= h:
            similarity = w*h/(c_w*c_h)
        elif c_w >= w and c_h <= h:
            similarity = w*c_h/(w*h + (c_w-w)*c_h)
        elif c_w <= w and c_h >= h:
            similarity = c_w*h/(w*h + c_w*(c_h-h))
        else: #means both w,h are bigger than c_w and c_h respectively
            similarity = (c_w*c_h)/(w*h)
        similarities.append(similarity) # will become (k,) shape

    return np.array(similarities)

def avg_IOU(anns, centroids):
    n,d = anns.shape
    sum = 0.

    for i in range(anns.shape[0]):
        sum+= max(IOU(anns[i], centroids))

    return sum/n

def print_anchors(centroids):
    anchors = centroids.copy()

    widths = anchors[:, 0]
    sorted_indices = np.argsort(widths)

    r = "anchors: ["
    for i in sorted_indices[:-1]:
        r += '%0.2f,%0.2f, ' % (anchors[i,0], anchors[i,1])

    #there should not be comma after last anchor, that's why
    r += '%0.2f,%0.2f' % (anchors[sorted_indices[-1:],0], anchors[sorted_indices[-1:],1])
    r += "]"

    print(r)

def run_kmeans(ann_dims, anchor_num):
    ann_num = ann_dims.shape[0]
    iterations = 0
    prev_assignments = np.ones(ann_num)*(-1)
    iteration = 0
    old_distances = np.zeros((ann_num, anchor_num))

    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
    centroids = ann_dims[indices]
    anchor_dim = ann_dims.shape[1]

    while True:
        distances = []
        iteration += 1
        for i in range(ann_num):
            d = 1 - IOU(ann_dims[i], centroids)
            distances.append(d)
        distances = np.array(distances) # distances.shape = (ann_num, anchor_num)

        print("iteration {}: dists = {}".format(iteration, np.sum(np.abs(old_distances-distances))))

        #assign samples to centroids
        assignments = np.argmin(distances,axis=1)

        if (assignments == prev_assignments).all() :
            return centroids

        #calculate new centroids
        centroid_sums=np.zeros((anchor_num, anchor_dim), np.float)
        for i in range(ann_num):
            centroid_sums[assignments[i]]+=ann_dims[i]
        for j in range(anchor_num):
            centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)

        prev_assignments = assignments.copy()
        old_distances = distances.copy()

def main():

    num_anchors = BOX

#     with open(config_path) as config_buffer:
#         config = json.loads(config_buffer.read())

    train_imgs, train_labels = parse_annotation(train_annot_folder, train_image_folder, labels=LABELS)

    grid_w = GRID_W
    grid_h = GRID_H

    # run k_mean to find the anchors
    annotation_dims = []
    for image in train_imgs:
        cell_w = image['width']/grid_w
        cell_h = image['height']/grid_h

        for obj in image['object']:
            relative_w = (float(obj['xmax']) - float(obj['xmin']))/cell_w
            relatice_h = (float(obj["ymax"]) - float(obj['ymin']))/cell_h
            annotation_dims.append(tuple(map(float, (relative_w,relatice_h))))

    annotation_dims = np.array(annotation_dims)
    centroids = run_kmeans(annotation_dims, num_anchors)

    # write anchors to file
    print('\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
    print_anchors(centroids)

if __name__ == '__main__':
    main()

iteration 1: dists = 34258.5942900603
iteration 2: dists = 3713.266475702192
iteration 3: dists = 1809.6983019829256
iteration 4: dists = 1304.0162964805763
iteration 5: dists = 927.1580658768173
iteration 6: dists = 645.5687949130177
iteration 7: dists = 457.499596399006
iteration 8: dists = 322.03568950909454
iteration 9: dists = 324.01433684313935
iteration 10: dists = 268.8813512921743
iteration 11: dists = 278.1167692046097
iteration 12: dists = 205.87080292833343
iteration 13: dists = 182.12126996260443
iteration 14: dists = 188.64189649103514
iteration 15: dists = 157.23845782364413
iteration 16: dists = 142.27824044150594
iteration 17: dists = 131.55334041356895
iteration 18: dists = 106.66399548601406
iteration 19: dists = 84.97223571419342
iteration 20: dists = 77.19267830259068
iteration 21: dists = 65.12312687445151
iteration 22: dists = 58.0971197896362
iteration 23: dists = 58.68603304449317
iteration 24: dists = 36.89604619029725
iteration 25: dists = 39.65402830898277
i